<a href="https://colab.research.google.com/github/ArifAygun/Webscraping/blob/main/Analysis_Freedom_Debt_Relief_Part_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SENTIMENT ANALYSIS OF FREEDOM DEPT RELIEF COMPANY**
# **REVIEWS IN TRUSTPILOT WEBSITE**

In [3]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
from torch.utils.data import  random_split, DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Freedom Debt Relief/

Mounted at /content/drive/
/content/drive/My Drive/Freedom Debt Relief


In [5]:
df= pd.read_csv('Freedom_Debt_Relief3.csv')

In [6]:
# Drop rows with null values in the 'Cleaned_Concatenated_Text' column
df = df.dropna(subset=['Cleaned_Concatenated_Text'])
# Display information after dropping null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33452 entries, 0 to 33453
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Reviewer Name              33452 non-null  object
 1   Reviews Count              33452 non-null  int64 
 2   Country Code               33452 non-null  object
 3   Experience Date            33452 non-null  object
 4   Rating                     33452 non-null  int64 
 5   Review Date                33452 non-null  object
 6   Review Title               33452 non-null  object
 7   Review Text                33452 non-null  object
 8   Reply Date                 31899 non-null  object
 9   Reply Text                 32133 non-null  object
 10  Country Name               33452 non-null  object
 11  Year                       33452 non-null  int64 
 12  Review Year                33452 non-null  int64 
 13  Review Response            33452 non-null  object
 14  Reply 

In [7]:
df.head().T

,0,1,2,3,4
Reviewer Name,Freida Towe,Jabrea Herron,William Lutz,John Novotny,Maria S
Reviews Count,1,1,2,1,1
Country Code,US,US,US,US,US
Experience Date,2019-12-04,2019-12-04,2019-12-04,2019-12-04,2019-12-04
Rating,5,5,5,5,1
Review Date,2019-12-04,2019-12-04,2019-12-04,2019-12-04,2019-12-04
Review Title,Very polite and kind,Great customer service,Needed an answer to a question…,i had a question,This company charges outrageous…
Review Text,Very polite and kind. Did what I asked immedia...,There wasn't a long wait time . The representa...,Needed an answer to a question concerning a pa...,I got my answer right away . very professional.,This company charges outrageous settlement fee...
Reply Date,2019-12-04,2019-12-05,2019-12-04,2019-12-04,2019-12-04
Reply Text,"Thank you very much, Freida! We truly care abo...","Jabrea, We are so happy to hear that you are h...","Hello William, thank you for sharing your expe...","John, thank you for your review. We strive to ...","We apologize, Ms. Maria, for any frustrations ..."


In [8]:
# Select a random sample of  data points from the entire DataFrame
df_sample = df.sample(n=3000, random_state=42)

# Reset index for the new sampled DataFrame
df_sample.reset_index(drop=True, inplace=True)

In [9]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Reviewer Name              5000 non-null   object
 1   Reviews Count              5000 non-null   int64 
 2   Country Code               5000 non-null   object
 3   Experience Date            5000 non-null   object
 4   Rating                     5000 non-null   int64 
 5   Review Date                5000 non-null   object
 6   Review Title               5000 non-null   object
 7   Review Text                5000 non-null   object
 8   Reply Date                 4764 non-null   object
 9   Reply Text                 4795 non-null   object
 10  Country Name               5000 non-null   object
 11  Year                       5000 non-null   int64 
 12  Review Year                5000 non-null   int64 
 13  Review Response            5000 non-null   object
 14  Reply Re

# Deep Learning Model (BERT)

## Loading Pre-trained BERT Tokenizer and Model:

In [10]:
# Load the pre-trained BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # Adjust num_labels based on your sentiment classes

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Load the pre-trained BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Load the pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # Adjust num_labels based on your sentiment classes

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenizing and Preprocessing:

In [12]:
max_seq_length = 128

# Tokenize and preprocess the text data
encoded_texts = tokenizer(list(df_sample['Cleaned_Concatenated_Text']), padding=True, truncation=True, max_length=max_seq_length, return_tensors='pt')

## Preparing Labels:

In [13]:
# Assuming you have 'Sentiment_Label' as your target variable
labels = torch.tensor(df_sample['Sentiment_Label'].map({'Negative': 0, 'Neutral': 1, 'Positive': 2}), dtype=torch.long)

## Splitting Data and Creating Data Loaders:

In [14]:
# Combine input_ids and attention_mask into a single tensor
input_tensor = torch.cat((encoded_texts['input_ids'].unsqueeze(2), encoded_texts['attention_mask'].unsqueeze(2)), dim=2)

# Create a TensorDataset
dataset = TensorDataset(input_tensor, labels)

# Define the sizes for training and validation sets
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size

# Split the dataset
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Create DataLoader for training and validation
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

## Defining Optimizer and Criterion:

In [15]:
# Define optimizer and criterion
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

## Training the Model (with graph):

In [ ]:
# Lists to store the loss values
epoch_losses = []
batch_losses = []

# Train the model
num_epochs = 2
model.train()

for epoch in range(num_epochs):
    total_loss = 0.0  # Variable to store the total loss for the epoch

    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        # Unpack the batch
        inputs, targets = batch

        # Split inputs tensor into input_ids and attention_mask
        input_ids, attention_mask = inputs[:, :, 0], inputs[:, :, 1]

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)[0]

        # Calculate loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()  # Accumulate the loss for the epoch
        batch_losses.append(loss.item())

        # Print loss after each batch
        print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Print average loss for the epoch
    average_loss = total_loss / len(train_loader)
    epoch_losses.append(average_loss)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {average_loss:.4f}')

# Plotting the training loss
plt.figure(figsize=(12, 6))

# Plot batch losses
plt.subplot(1, 2, 1)
plt.plot(batch_losses, label='Batch Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training Batch Loss Over Time')
plt.legend()

# Plot epoch average losses
plt.subplot(1, 2, 2)
plt.plot(epoch_losses, label='Epoch Average Loss', linestyle='--', marker='o', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Epoch Average Loss Over Time')
plt.legend()

plt.tight_layout()
plt.show()

Epoch [1/3], Batch [1/250], Loss: 1.0825
Epoch [1/3], Batch [2/250], Loss: 1.0144
Epoch [1/3], Batch [3/250], Loss: 0.9231
Epoch [1/3], Batch [4/250], Loss: 0.8497
Epoch [1/3], Batch [5/250], Loss: 0.8696
Epoch [1/3], Batch [6/250], Loss: 0.6814
Epoch [1/3], Batch [7/250], Loss: 0.6692
Epoch [1/3], Batch [8/250], Loss: 0.6216
Epoch [1/3], Batch [9/250], Loss: 0.7333
Epoch [1/3], Batch [10/250], Loss: 0.6390
Epoch [1/3], Batch [11/250], Loss: 0.5415
Epoch [1/3], Batch [12/250], Loss: 0.4370
Epoch [1/3], Batch [13/250], Loss: 0.2661
Epoch [1/3], Batch [14/250], Loss: 0.6189
Epoch [1/3], Batch [15/250], Loss: 0.4545
Epoch [1/3], Batch [16/250], Loss: 0.3234
Epoch [1/3], Batch [17/250], Loss: 0.1785
Epoch [1/3], Batch [18/250], Loss: 0.4460
Epoch [1/3], Batch [19/250], Loss: 0.1478
Epoch [1/3], Batch [20/250], Loss: 0.7911
Epoch [1/3], Batch [21/250], Loss: 0.4508
Epoch [1/3], Batch [22/250], Loss: 0.2775
Epoch [1/3], Batch [23/250], Loss: 0.1221
Epoch [1/3], Batch [24/250], Loss: 0.8030
E

## Evaluating the Model:

In [ ]:
model.eval()
val_predictions = []
val_targets = []

with torch.no_grad():
    for input_batch, target_batch in valid_loader:
        # Assuming the structure of your input_batch is [input_ids, attention_mask]
        # and the structure of your target_batch is [targets]
        input_ids, attention_mask = input_batch[:, :, 0], input_batch[:, :, 1]
        targets = target_batch

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)[0]
        _, predicted_labels = torch.max(outputs, 1)
        val_predictions.extend(predicted_labels.cpu().numpy())
        val_targets.extend(targets.cpu().numpy())

# Print classification report
print(classification_report(val_targets, val_predictions))

# Create a confusion matrix
cm = confusion_matrix(val_targets, val_predictions)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1', 'Class 2'], yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## Calculating Metrics:

In [ ]:
# Calculate accuracy and other metrics
accuracy = accuracy_score(val_targets, val_predictions)
f1 = f1_score(val_targets, val_predictions, average='weighted')
classification_rep = classification_report(val_targets, val_predictions)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_rep)

# Additional plot for the classification report
report_data = classification_report(val_targets, val_predictions, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# Print precision, recall, and f1-score values for each class
for index, row in report_df.iterrows():
    print(f"\nClass {index} - Precision: {row['precision']:.3f}, Recall: {row['recall']:.3f}, F1-Score: {row['f1-score']:.3f}")

# Print accuracy
print(f"\nAccuracy: {accuracy:.4f}")

In [ ]:
# Calculate accuracy and other metrics
accuracy = accuracy_score(val_targets, val_predictions)
f1 = f1_score(val_targets, val_predictions, average='weighted')
classification_rep = classification_report(val_targets, val_predictions)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_rep)

# Additional plot for the classification report
report_data = classification_report(val_targets, val_predictions, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# Print precision, recall, and f1-score values for each class
for index, row in report_df.iterrows():
    print(f"\nClass {index} - Precision: {row['precision']:.3f}, Recall: {row['recall']:.3f}, F1-Score: {row['f1-score']:.3f}")

# Print accuracy
print(f"\nAccuracy: {accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(val_targets, val_predictions)

# Plotting Confusion Matrix as a Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1', 'Class 2'],
            yticklabels=['Class 0', 'Class 1', 'Class 2'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Visualize True Positive, True Negative, False Positive, False Negative
tp = np.diag(conf_matrix)
fp = np.sum(conf_matrix, axis=0) - tp
fn = np.sum(conf_matrix, axis=1) - tp
tn = np.sum(conf_matrix) - (tp + fp + fn)

# Plotting True Positive, True Negative, False Positive, False Negative
plt.figure(figsize=(10, 6))
barWidth = 0.2
r1 = np.arange(len(tp))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

plt.bar(r1, tp, color='green', width=barWidth, edgecolor='grey', label='True Positive')
plt.bar(r2, tn, color='blue', width=barWidth, edgecolor='grey', label='True Negative')
plt.bar(r3, fp, color='red', width=barWidth, edgecolor='grey', label='False Positive')
plt.bar(r4, fn, color='orange', width=barWidth, edgecolor='grey', label='False Negative')

plt.xlabel('Classes', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(tp))], ['Class 0', 'Class 1', 'Class 2'])
plt.title('True Positive, True Negative, False Positive, False Negative')
plt.legend()
plt.show()